In [4]:

import os, warnings

from tensorflow_hub.keras_layer import keras

os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
import matplotlib.pyplot as plt
from matplotlib import gridspec

import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory


# reproducibility
def set_seed(seed=31415):
    np.random.seed(seed)
    tf.random.set_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    os.environ['TF_DETERMINISTIC_OPS'] = '1'


set_seed()

plt.rc('figure', autolayout=True)
plt.rc('axes', labelweight='bold', labelsize='large', titleweight='bold', titlesize=18, titlepad=10)
plt.rc('image', cmap='gray')
warnings.filterwarnings('ignore')

# load training and load data
ds_train_ = image_dataset_from_directory(
    './data/car_or_truck/train',
    labels='inferred',
    shuffle=True,
    label_mode='binary',
    image_size=[128, 128],
    interpolation='nearest',
    batch_size=64,
)

ds_valid_ = image_dataset_from_directory(
    './data/car_or_truck/valid',
    labels='inferred',
    shuffle=True,
    label_mode='binary',
    image_size=[128, 128],
    interpolation='nearest',
    batch_size=64,
)


# Datapipeline
def convert_to_float_image(image, label):
    image = tf.cast(image, tf.float32)
    return image, label


AUTOTUNE = tf.data.experimental.AUTOTUNE
ds_train = (
    ds_train_
    .map(convert_to_float_image)
    .cache()
    .prefetch(buffer_size=AUTOTUNE)
)

ds_valid = (
    ds_valid_
    .map(convert_to_float_image)
    .cache()
    .prefetch(buffer_size=AUTOTUNE)
)

Found 5117 files belonging to 2 classes.
Found 5051 files belonging to 2 classes.


In [6]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Load pretrained VGG16 base model (saved locally)
pretrained_base = tf.keras.models.load_model(
    './models/cv-course-models/vgg16-pretrained-base'
)
pretrained_base.trainable = False

# Define model
model = keras.Sequential([
    # ✅ Preprocessing layers
    layers.RandomFlip('horizontal'),       # flip left to right
    layers.RandomContrast(0.5),            # adjust contrast

    # ✅ Base (pretrained)
    pretrained_base,

    # ✅ Head
    layers.Flatten(),
    layers.Dense(6, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

model.compile(
    optimizer=keras.optimizers.Adam(),
    loss='binary_crossentropy',
    metrics=['binary_accuracy'],
)

history = model.fit(
    ds_train,
    evalidation_data=ds_valid,
    epochs=10,
    v
)


TypeError: Exception encountered when calling TFSMLayer.call().

[1mCould not automatically infer the output shape / dtype of 'tfsm_layer_2' (of type TFSMLayer). Either the `TFSMLayer.call()` method is incorrect, or you need to implement the `TFSMLayer.compute_output_spec() / compute_output_shape()` method. Error encountered:

Binding inputs to tf.function failed due to `too many positional arguments`. Received args: (<tf.Tensor 'Placeholder:0' shape=(None, 224, 224, 3) dtype=float32>,) and kwargs: {} for signature: (*, input_1: TensorSpec(shape=(None, 128, 128, 3), dtype=tf.float32, name='input_1')) -> Dict[['block5_pool', TensorSpec(shape=(None, 4, 4, 512), dtype=tf.float32, name='block5_pool')]].
Fallback to flat signature also failed due to: Tensor Tensor("Placeholder:0", shape=(None, 224, 224, 3), dtype=float32) is not compatible with the shape this function was traced with. Expected shape (None, 128, 128, 3), but got shape (None, 224, 224, 3).

If you called get_concrete_functio